#  数据规整化：清洗、转换、合并、重塑

###  数据库风格的DataFrame合并

In [1]:
from pandas import DataFrame,Series

In [2]:
import pandas as pd

In [3]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [10]:
df1 = DataFrame({'key':['b','b','a','c','a','a','b'],
                'data1':range(7)})

In [11]:
df2 = DataFrame({'key':['a', 'b','d'], 'data2':range(3)})

In [12]:
df1

,data1,key
0,0,b
1,1,b
2,2,a
3,3,c
4,4,a
5,5,a
6,6,b


In [13]:
df2

,data2,key
0,0,a
1,1,b
2,2,d


In [14]:
pd.merge(df1, df2)

,data1,key,data2
0,0,b,1
1,1,b,1
2,6,b,1
3,2,a,0
4,4,a,0
5,5,a,0


In [15]:
pd.merge(df1, df2, on='key')

,data1,key,data2
0,0,b,1
1,1,b,1
2,6,b,1
3,2,a,0
4,4,a,0
5,5,a,0


### 如果两个对象的列名不同，也可以分别进行指定

In [18]:
df3 = DataFrame({'lkey':['b','b','a','c','a','a','b'],
                'data1':range(7)})

In [20]:
df4 = DataFrame({'rkey':['a','b','d'],
                         'data2':range(3)})

In [21]:
df3

,data1,lkey
0,0,b
1,1,b
2,2,a
3,3,c
4,4,a
5,5,a
6,6,b


In [22]:
df4

,data2,rkey
0,0,a
1,1,b
2,2,d


In [23]:
pd.merge(df3, df4, left_on='lkey',right_on='rkey')

,data1,lkey,data2,rkey
0,0,b,1,b
1,1,b,1,b
2,6,b,1,b
3,2,a,0,a
4,4,a,0,a
5,5,a,0,a


### 默认情况下，merge做的是“inner”连接，结果中的键是交集。外连接取的是键的bingji：

In [24]:
pd.merge(df1, df2, how='outer')

,data1,key,data2
0,0.0,b,1.0
1,1.0,b,1.0
2,6.0,b,1.0
3,2.0,a,0.0
4,4.0,a,0.0
5,5.0,a,0.0
6,3.0,c,NaN
7,NaN,d,2.0


In [25]:
# 多对多的合并操作

In [26]:
df1 = DataFrame({'key':['b','b','a','c','a','b'],
                'data1': range(6)})

In [27]:
df2 = DataFrame({'key':['a', 'b', 'a','b','d'],
                'data2': range(5)})

In [28]:
df1

,data1,key
0,0,b
1,1,b
2,2,a
3,3,c
4,4,a
5,5,b


In [29]:
df2

,data2,key
0,0,a
1,1,b
2,2,a
3,3,b
4,4,d


In [35]:
df11 = pd.merge(df1, df2, on='key', how='left')
# 多对多连接产生的是行的笛卡尔积，由于左边的df有3个‘b’行，右边的有2个，所以最终结果中有6个‘b’行
# 连接方式只影响出现在结果中的键

In [36]:
df11.sort_values('key')

,data1,key,data2
4,2,a,0.0
5,2,a,2.0
7,4,a,0.0
8,4,a,2.0
0,0,b,1.0
1,0,b,3.0
2,1,b,1.0
3,1,b,3.0
9,5,b,1.0
10,5,b,3.0


In [39]:
pd.merge(df1, df2, how='inner').sort_values('key')

,data1,key,data2
6,2,a,0
7,2,a,2
8,4,a,0
9,4,a,2
0,0,b,1
1,0,b,3
2,1,b,1
3,1,b,3
4,5,b,1
5,5,b,3


###  要根据多个键进行合并，传入一个由列名组成的列表即可

In [47]:
left = DataFrame({'key1':['foo','foo','bar'],
                 'key2': ['one','two','one'],
                 'lval':[1, 2, 3]})

In [48]:
right = DataFrame({'key1':['foo','foo','bar','bar'],
                  'key2':['one','one','one','two'],
                  'rval':[4,5,6,7]})

In [49]:
left

,key1,key2,lval
0,foo,one,1
1,foo,two,2
2,bar,one,3


In [50]:
right

,key1,key2,rval
0,foo,one,4
1,foo,one,5
2,bar,one,6
3,bar,two,7


In [51]:
pd.merge(left, right, on=['key1', 'key2'], how='outer')

,key1,key2,lval,rval
0,foo,one,1.0,4.0
1,foo,one,1.0,5.0
2,foo,two,2.0,NaN
3,bar,one,3.0,6.0
4,bar,two,NaN,7.0


### 在进行列-列连接时，DataFrame对象中的索引会被丢弃

In [52]:
pd.merge(left, right, on='key1')

,key1,key2_x,lval,key2_y,rval
0,foo,one,1,one,4
1,foo,one,1,one,5
2,foo,two,2,one,4
3,foo,two,2,one,5
4,bar,one,3,one,6
5,bar,one,3,two,7


In [53]:
pd.merge(left, right, on='key1', suffixes=('_left', '_right'))

,key1,key2_left,lval,key2_right,rval
0,foo,one,1,one,4
1,foo,one,1,one,5
2,foo,two,2,one,4
3,foo,two,2,one,5
4,bar,one,3,one,6
5,bar,one,3,two,7


### 索引上的合并

#### 有时候连接键位于索引中，此时可以传入lefty_index=True或right_index=True（或两个都传）以说明索引应该被用作连接键：

In [54]:
left1 = DataFrame({'key':['a','b','a','a','b','c'],
                  'value':range(6)})

In [55]:
right1 = DataFrame({'group_val':[3.5, 7]}, index={'a', 'b'})

In [56]:
left1

,key,value
0,a,0
1,b,1
2,a,2
3,a,3
4,b,4
5,c,5


In [57]:
right1

,group_val
a,3.5
b,7.0


In [58]:
pd.merge(left1, right1, left_on='key', right_index=True)

,key,value,group_val
0,a,0,3.5
2,a,2,3.5
3,a,3,3.5
1,b,1,7.0
4,b,4,7.0


In [59]:
pd.merge(left1, right1, left_on='key', right_index=True, how='outer')

,key,value,group_val
0,a,0,3.5
2,a,2,3.5
3,a,3,3.5
1,b,1,7.0
4,b,4,7.0
5,c,5,NaN


In [61]:
lefth = DataFrame({'key1':['Ohio','Ohio','Ohio','Nevada','Nevada'],
                  'key2':[2000, 2001, 2002, 2001, 2002],
                  'data': np.arange(5.)})

In [69]:
righth = DataFrame(np.arange(12).reshape((6,2)), index=[['Nebada','Nevada','Ohio','Ohio','Ohio','Ohio'],
                                                       [2001,2000,2000,2000,2001,2002]],
                                                       columns = ['event1','event2'])

In [70]:
lefth

,data,key1,key2
0,0.0,Ohio,2000
1,1.0,Ohio,2001
2,2.0,Ohio,2002
3,3.0,Nevada,2001
4,4.0,Nevada,2002


In [71]:
righth

event1  event2
Nebada 2001       0       1
Nevada 2000       2       3
Ohio   2000       4       5
       2000       6       7
       2001       8       9
       2002      10      11

In [72]:
pd.merge(lefth, righth, left_on=['key1','key2'], right_index=True)

,data,key1,key2,event1,event2
0,0.0,Ohio,2000,4,5
0,0.0,Ohio,2000,6,7
1,1.0,Ohio,2001,8,9
2,2.0,Ohio,2002,10,11


In [73]:
pd.merge(lefth, righth, left_on=['key1', 'key2'],
        right_index=True, how='outer')

,data,key1,key2,event1,event2
0,0.0,Ohio,2000,4.0,5.0
0,0.0,Ohio,2000,6.0,7.0
1,1.0,Ohio,2001,8.0,9.0
2,2.0,Ohio,2002,10.0,11.0
3,3.0,Nevada,2001,NaN,NaN
4,4.0,Nevada,2002,NaN,NaN
4,NaN,Nebada,2001,0.0,1.0
4,NaN,Nevada,2000,2.0,3.0


In [74]:
left2 = DataFrame([[1., 2.], [3., 4.], [5., 6.]], index=['a','c','e'],
                 columns=['Ohio', 'Nevada'])

In [76]:
right2 = DataFrame([[7., 8.], [9., 10.],[11., 12.],[13., 14.]],
                  index=['b','c','d','e'],
                  columns=['Missouri','Alabana'])

In [77]:
left2

,Ohio,Nevada
a,1.0,2.0
c,3.0,4.0
e,5.0,6.0


In [78]:
right2

,Missouri,Alabana
b,7.0,8.0
c,9.0,10.0
d,11.0,12.0
e,13.0,14.0


In [79]:
pd.merge(left2, right2, how='outer', left_index=True, right_index=True)

,Ohio,Nevada,Missouri,Alabana
a,1.0,2.0,NaN,NaN
b,NaN,NaN,7.0,8.0
c,3.0,4.0,9.0,10.0
d,NaN,NaN,11.0,12.0
e,5.0,6.0,13.0,14.0
